<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Dataframes III
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Обзор источников данных
+ Текстовые форматы txt, csv, json
+ Parquet и ORC
+ Elastic
+ Cassandra
+ PostgreSQL

## Обзор источников данных
Spark - это платформа для **обработки** распределенных данных. Она не отвечает за хранение данных и не завязана на какую-либо БД или формат хранения, что позволяет разработать коннектор для работы с любым источником. Часть распространенных источников доступна "из коробки", часть - в виде сторонних библиотек. 

На текущий момент Spark DF API позволяет работать (читать и писать) с большим набором источников:
+ Текстовые файлы:
  - [json](https://spark.apache.org/docs/latest/sql-data-sources-json.html)
  - text
  - [csv](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.csv)
+ Бинарные файлы:
  - [orc](https://spark.apache.org/docs/latest/sql-data-sources-orc.html)
  - [parquet](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html)
  - [delta](https://docs.delta.io/latest/quick-start.html)
+ Базы данных
  - [elastic](https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html#spark-sql)
  - [cassandra](https://github.com/datastax/spark-cassandra-connector/blob/master/doc/14_data_frames.md)
  - [jdbc](https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html)
  - [redis](https://github.com/RedisLabs/spark-redis/blob/master/doc/dataframe.md)
  - [mongo](https://docs.mongodb.com/spark-connector/master/scala-api/)
+ Стриминг системы
  - [kafka](https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html)

Для текстовых файлов поддерживаются различные кодеки сжатия (например `lzo`, `snappy`, `gzip`)

### Добавление поддержки
Чтобы добавить поддержку источника в проект, необходимо:
+ найти нужный пакет на https://mvnrepository.com
 - выбрать актуальную версию для `Scala 2.11`
 - скачать jar или скопировать команду для нужной системы сборки 
+ добавить зависимость в `libraryDependencies` в файле `build.sbt`  
```libraryDependencies += "org.elasticsearch" %% "elasticsearch-spark-20" % "7.7.0"```
+ добавить зависимость в приложение одним из способов:
  - добавить зависимость в **spark-submit**:  
  ```spark-submit --packages org.elasticsearch:elasticsearch-spark-20_2.11:7.7.0```
  - добавить jar файл в **spark-submit**:  
  ```spark-submit --jars /path/to/elasticsearch-spark-20_2.11-7.7.0.jar```
  - добавить зависимость в **spark-defaults.conf**:  
  ```spark.jars.packages org.elasticsearch:elasticsearch-spark-20_2.11:7.7.0```
  - добавить jar файл в **spark-defaults.conf**:  
  ```spark.jars /path/to/elasticsearch-spark-20_2.11-7.7.0.jar```
  - в коде через [`spark.sparkContext.addJar`](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.SparkContext@addJar(path:String):Unit)
  
### Использование в коде
Конфиги источника задаются одним из способов:
- через **spark-submit**:  
```spark-submit --conf spark.es.nodes=localhost:9200```
- в **spark-defaults.conf**:  
```spark.es.nodes localhost:9200```
- в коде через **SparkSession**:
  + ```spark.conf.set("spark.es.nodes", "localhost:9200")```
- в коде при чтении:  
  + ```val df = spark.read.format("elastic").option("es.nodes", "localhost:9200")...```
  + ```val df = spark.read.format("elastic").options(Map("es.nodes" -> "localhost:9200"))...```
- в коде при записи:  
  + ```df.write.format("elastic").option("es.nodes", "localhost:9200")...```
  + ```df.write.format("elastic").options(Map("es.nodes" -> "localhost:9200"))...```
  
### Выводы:
- Spark позволяет работать с болшим количеством источников
- Поддержка источника всегда добавлеяется на уровне JVM (даже для pyspark) путем добавления в `java classpath` нужного класса
- Добавить поддержку источника можно по-разному, однако в большинстве случаев следует избегать "хардкода"

## Текстовые форматы

Spark позволяет хранить данные в текстовом виде в форматах `text`, `json`, `csv`
- `json` - JSON строки (не массив JSON документов, а именно раздельные строки, разделенные `\n`)  
- `csv` - плоские данные с разделителем  
- `text` просто текстовые строки, вычитываются как DF с единственной колонкой `value: String`  

### Преимущества:
- простота интеграции
- поддержка партиционирования и сжатия

### Недостатки:
- отсутствие оптимизаций 
- низкая скорость чтения сжатых данных
- слабая типизация

Прочитаем датасет [Airport Codes](https://datahub.io/core/airport-codes):

In [ ]:
import sys.process._

println("ls -al /tmp/datasets/airport-codes.csv".!!)

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Запишем его в формате `csv`. Запись данных происходит в директорию, внутри которой будут файлы с данными. Это свойство является общим для всех файловых форматов в Spark

In [ ]:
airports.write.mode("overwrite").csv("/tmp/datasets/airports-2.csv")

In [ ]:
println("ls -al /tmp/datasets/airports-2.csv".!!)

Если мы попытаемся прочитать его с помощью `spark.read`, используя старый код, получим ошибку - в качестве схемы Spark взял одну из строк, содержащую данные.

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airports-2.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Поищем шапку в сырых данных - ее там не будет:

In [ ]:
spark.read.text("/tmp/datasets/airports-2.csv").filter('value contains "elevation_ft").count

Если прочитать с `header=false`, названия колонок будут автоматически сгенерированы:

In [ ]:
val csvOptions = Map("header" -> "false", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airports-2.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Имея шапку в виде строки, мы можем создать схему самостоятельно:

In [ ]:
import org.apache.spark.sql.types._

val firstLine = "head -n 1 /tmp/datasets/airport-codes.csv".!!
val schema = StructType(firstLine.split(",", -1).map(x => StructField(x, StringType)))
val csvOptions = Map("header" -> "false", "inferSchema" -> "true")
val airports = spark.read.schema(schema).options(csvOptions).csv("/tmp/datasets/airports-2.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Сохраним данные в `csv` с включенной компрессией `gzip`:

In [ ]:
airports.write.mode("overwrite").option("codec", "gzip").csv("/tmp/datasets/airports-3.csv")

In [ ]:
println("ls -alh /tmp/datasets/airports-3.csv".!!)

Данные стали занимать меньше места, но у этого решения есть существенный минус - при чтении каждый сжатый файл превращается ровно в 1 партицию в DF. При работе с большими датасетами это означает:
- если файлов мало и они большие, то воркерам может не хватить памяти для их чтения (тк один сжатый файл нельзя разбить на несколько партиций
- если файлов много и они маленькие - мы получаем увеличенный расход памяти в heap HDFS NameNode (память расходуется пропорционально количеству файлов на HDFS из расчета 1 ГБ памяти на 1 000 000 файлов)

Сохраним датасет в формате `json` с партиционирование по колонкам `iso_region` и `iso_country`:

In [ ]:
airports.write.mode("overwrite").partitionBy("iso_region", "iso_country").json("/tmp/datasets/airports-1.json")

In [ ]:
println("ls -alh /tmp/datasets/airports-1.json/iso_region=AD-04/iso_country=AD".!!)

Такой формат хранения позволит использовать `partition pruning` и быстро фильтровать данные по колонкам `iso_region` и `iso_country`

Теперь сохраним датасет в формат `text`. Для этого нам необходимо подгтовить DF, в котором будет единственная колонка `value: String`

In [ ]:
airports
    .select('ident.alias("value"))
    .write
    .mode("overwrite")
    .format("text")
    .save("/tmp/datasets/airports-1.txt")

In [ ]:
println("ls -alh /tmp/datasets/airports-1.txt".!!)

Файловые форматы не имеют автоматической валидации данных при записи, поэтому достаточно легко ошибиться и записать данные в другом формате. Такая запись пройдет без ошибок:

In [ ]:
airports
    .write
    .mode("append")
    .json("/tmp/datasets/airports-1.txt")

При попытке чтения данных с помощью `text` мы получим все данные, тк форма `json` сохраняет все в виде JSON строк. Однако, если прочитать данные с помощью `json`, часть данных будут помечены как невалидные и помещены в колонку `_corrupt_record`

In [ ]:
val airports = spark.read.json("/tmp/datasets/airports-1.txt")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Отобразим невалидные JSON строки:

In [ ]:
// Начиная со Spark 2.3 нельзя выбирать одну колонку _corrupt_record, поэтому мы добавим к выводу ident
airports.na.drop("all", Seq("_corrupt_record")).select($"_corrupt_record", $"ident").show(20, false)

### Режимы записи
Spark позволяет нам выбирать режим записи данных с помощью метода `mode()`. Данный метод принимает один из параметров:
- `overwrite` - перезаписывает всю директорию целиком (или партицию, если используется партиционирование)
- `append` - дописывает новые файлы к текущим
- `ignore` - не выполняет запись (no op режим)
- `error` или `errorifexists` - возвращает ошибку, если директория уже существует

### Семплирование

Форматы `csv` и `json` позволяют автоматически выводить схему из данных. При этом по-умолчанию Spark прочитает все данные и составит подходящую схему. Однако, если мы работаем с большим датасетом, это может занять продолжительное время. Решить это можно с помощью опции `samplingRatio`:

In [ ]:
spark.time { 
    val csvOptions = Map("header" -> "true", "inferSchema" -> "true", "samplingRatio" -> "0.1")
    val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")
    airports.printSchema
}

In [ ]:
spark.time { 
    val csvOptions = Map("header" -> "true", "inferSchema" -> "true", "samplingRatio" -> "1.0")
    val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")
    airports.printSchema
}

### Выводы
- Spark позволяет работать с текстовыми файлами `json`, `csv`, `text`
- При чтении и записи поддерживаются кодеки сжатия данных, это создает дополнительные накладные расходы
- При записи данных в текстовые форматы Spark **не выполняет** валидацию схемы и формата
- При включенном выведении схемы из источника чтение из текстовых форматов происходит дольше

## Orc и Parquet
В отличие от обычных текстовых форматов, ORC и Parquet изначально спроектированы под распределенные системы хранения и обработки. Они являются колоночными - в них есть колонки и схема, как в таблицах БД и бинарными - прочитать обычным текстовым редактором их не получится. Форматы имеют похожие показатели производительности и архитектуру, но Parquet используется чаще

### Преимущества
- наличие схемы данных
- блочная компрессия 
- для каждого блока для каждой колонки вычисляется max и min, что позволяет ускорять чтение

### Недостатки:
- нельзя дописывать/менять данные в существующих файлах
- необходимо делать compaction

Подробнее о Parquet:  
[Фёдор Лаврентьев, Moscow Spark #5: Как класть Parquet](https://youtu.be/VHsvr10b63c?t=512)

По аналогии с текстовыми форматами, при записи, Spark создает директорию и пишет туда все непустые партиции.  Обратите внимание на последовательность форматов записи - `snappy.parquet` вместо, скажем, `json.gz`. При использовании компрессии сам parquet файл не помещается в сжатый контейнер. Вместо этого, компрессии подлежат блоки с данными. Это полностью снимает ограничение, из-за которого чтение сжатых текстовых файлов происходит в 1 поток в 1 партицию.

In [ ]:
airports.write.mode("overwrite").parquet("/tmp/datasets/airports-1.parquet")

In [ ]:
println("ls -alh /tmp/datasets/airports-1.parquet".!!)

In [ ]:
spark.read.parquet("/tmp/datasets/airports-1.parquet").printSchema

### Schema evolution

При работе с ORC/Parquet, часто возникает вопрос эволюции схемы - изменения структуры данных относительно первоначальных файлов. Создадим два DF с разными схемами и запишем их в одну директорию:

In [ ]:
case class Apple(size: Int, color: String)

In [ ]:
List(Apple(1, "green")).toDS.write.mode("append").parquet("/tmp/datasets/apples.parquet")

In [ ]:
case class PriceApple(size: Int, color: String, price: Double)

In [ ]:
List(PriceApple(1, "green", 2.0)).toDS.write.mode("append").parquet("/tmp/datasets/apples.parquet")

Несмотря на то, что файлы имеют разную схему, Spark корректно читает файлы, используя обобщенную схему:

In [ ]:
val df = spark.read.parquet("/tmp/datasets/apples.parquet")
df.show

Однако, это работает только тогда, когда мы добавляем новые колонки к нашей схеме. Если мы запишем новый файл, изменив тип уже существующей колонки, мы получим ошибку:

In [ ]:
case class AppleBase(size: Double)

In [ ]:
List(AppleBase(3.0)).toDS.write.mode("append").parquet("/tmp/datasets/apples.parquet")

In [ ]:
val df = spark.read.parquet("/tmp/datasets/apples.parquet")
df.show

Посмотрим все доступные опции для работы с Parquet:

In [ ]:
%%dataframe --limit 20
spark.sql("SET -v").filter('key contains "parquet")

### Parquet Tools
Для диагностики и решения проблем, связанных с parquet, можно использовать утилиту `parquet-tools`

Она позволяет:
- получить схему файла
- вывести содержимое файла в консоль
- объединить несколько файлов в один

https://github.com/apache/parquet-mr/tree/master/parquet-tools

### Сравнение скорости обработки запросов

Подготовим датасеты:

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")

In [ ]:
1 to 100 foreach { x =>
    airports.repartition(1).write.mode("append").parquet("/tmp/datasets/a1.parquet")
    airports.repartition(1).write.mode("append").json("/tmp/datasets/a1.json")
    airports.repartition(1).write.mode("append").orc("/tmp/datasets/a1.orc")
}

In [ ]:
import org.apache.spark.sql.Dataset
case class DatasetFormat[T](ds: Dataset[T], format: String)

val datasets = 
    DatasetFormat(spark.read.orc("/tmp/datasets/a1.orc"), "orc") ::
    DatasetFormat(spark.read.parquet("/tmp/datasets/a1.parquet"), "parquet") ::
    DatasetFormat(spark.read.json("/tmp/datasets/a1.json"), "json") ::
    Nil

Сравним скорость работы фильтраци:

In [ ]:
datasets.foreach { x => 
    println(s"Running ${x.format}")
    spark.time {
        println(x.ds.filter($"iso_country" === "RU" and $"elevation_ft" > 300).count)
    }
}

Сравним скорость подсчета количества строк:

In [ ]:
datasets.foreach { x => 
    println(s"Running ${x.format}")
    spark.time {
        println(x.ds.count)
    }
}

### Выводы:
- Форматы `orc` и `parquet` позволяют эффективно работать со структурированными данными
- Производительность `orc` и `parquet` на порядок выше обычных текстовых файлов
- Данные форматы поддерживают сжатие на блочном уровне, что позволяет избегать проблем с многопоточным чтением
- Форматы поддерживают добавление новых колонок в схему, но не изменение текущих

## Elastic
Документориентированная распределенная база данных.

### Преимущества:
- Удобный графический интерфейс Kibana
- Полнотекстовый поиск по любым колонкам
- Встроенная поддержка timeseries
- Поддержка вложенных структур
- Возможность записи данных с произвольной схемой
- Возможность перезаписывать данные по ключу документа

### Недостатки:
- Ассиметричная архитектура
- Скорость записи ограничена самой медленным узлом
- Большие накладные расходы CPU на индексирование
- Ротация шардов не всегда проходит гладко

https://www.elastic.co

### Запуск в docker
https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html#docker-cli-run-dev-mode  
https://www.elastic.co/guide/en/kibana/current/docker.html#_running_kibana_on_docker_for_development  

### Spark connector
https://mvnrepository.com/artifact/org.elasticsearch/elasticsearch-spark-20  
https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html  

В Elastic есть несколько основных сущностей:

**Index** - представляет собой "таблицу с данными", если проводить аналогию с реляционными БД. Данные в elastic обычно хранятся в виде индексов, разбитых на сутки (foo-2020-05-30, foo-2020-05-29 и т. д.). У каждого документа в индексе есть ключ `_id` и может быть метка времени, по которой в Kibana строятся визуализации

**Template** - шаблон с параметрами, с которыми создается новый индекс. Пример шаблона представлен ниже:
```shell
PUT _template/airports
{
  "index_patterns": ["airports-*"],
  "settings": {
    "number_of_shards": 1
  },
  "mappings": {
    "_doc": {
      "dynamic": true,
      "_source": {
        "enabled": true
      },
      "properties": {
        "ts": {
          "type": "date",
          "format": "strict_date_optional_time||epoch_millis"
        }
      }
    }
  }
}
```
**Shard** - индексы в elastic делятся ~~почкованием~~ на шарды. Это позволяет хранить индекс на нескольких узлах кластера

**Index Pattern** - шаблон, применяемый к индексу на уровне Kibana. Позволяет настраивать форматирование и подсветку полей

Перед тем, как начать писать в elastic с помощью Spark, нам необходимо создать шаблон, иначе индекс будет создан с параметрами по умолчанию и построить красивый pie chart в Kibana у нас не получится. Это можно сделать с помощью Dev Tools в Kibana.

In [ ]:
import org.apache.spark.sql.functions._

val esOptions = 
    Map(
        "es.nodes" -> "localhost:9200", 
        "es.batch.write.refresh" -> "false",
        "es.nodes.wan.only" -> "true"   
    )

airports
    .withColumn("ts", current_timestamp())
    .withColumn("date", current_date())
    .withColumn("foo", lit("foo"))
    .write.format("es").options(esOptions).save("airports-{date}/_doc")

In [ ]:
val esDf = spark.read.format("es").options(esOptions).load("airports-*")
esDf.printSchema
esDf.show(1, 200, true)

Количество партций в DF совпадает с общим числом шардов индексов, которые мы указали в `load()`. Поскольку у нас 1 индекс и 1 шард (см. шаблон), данный DF имеет 1 партицию:

In [ ]:
esDf.rdd.getNumPartitions

К применяемым фильтрам применяется оптимизация `filter pushdown`

In [ ]:
esDf.filter('iso_region contains "RU").explain(true)

In [ ]:
esDf
    .filter(
        'ts between (
                    lit("2020-06-01 16:57:30.000").cast("timestamp"), 
                    lit("2020-06-01 16:59:30.000").cast("timestamp")
        )
    ).explain(true)

### Выводы:
- Elastic - удобное распределенное хранилище документов, не накладывающее строгих ограничений на схему документов
- Elastic позволяет делать сложные запросы, включая полнотекстовые
- При работе с elastic, Spark часто использует `filter pushdown`
- Spark отлично подходит для того, чтобы писать в elastic. Однако чтение работает не очень быстро.

## Cassandra
Распределенная табличная база данных

### Преимущества
- Высокая доступность данных
- Мжожно строить гео-кластера
- Высокая скорость записи и чтения
- Скорость ограничена самым быстрым узлом
- Линейная масштабируемость
- Возможность хранить БОЛЬШИЕ объемы данных
- Возможность быстро получать строку по ключу на любом объеме данных

### Недостатки
- слабая согласованность (eventual)
- Бедный SQL (в кассандре он называется CQL)
- Отсутствие транзакций (не совсем)

https://cassandra.apache.org

Cassandra имеет симметричную архитектуру. Каждый узел отвечает за хранение данных, обработку запросов и состояние кластера. 

Расположение данных определяется значением хеш функции от Partition key.

Высокая доступность данных обеспечивается за счет репликации.

![Cassandra Architecture](https://cassandra.apache.org/doc/latest/_images/ring.svg)
Источник: https://cassandra.apache.org/doc/latest/architecture/dynamo.html#dataset-partitioning-consistent-hashing

### Запуск в docker

Запуск инстанса:
```shell
docker run --rm --name cass -p 9042:9042 -e CASSANDRA_BROADCAST_ADDRESS=127.0.0.1 cassandra:latest
```

Подключение к cassandra:
```shell
docker run -it --rm cassandra:latest cqlsh host.docker.internal
```

В Cassandra есть:
- `keyspace` - аналог database - логическое объединение таблиц. На уровне keyspace устанавливается фактор репликации
- `table` - таблицы, как в обычной БД

### Spark connector
https://mvnrepository.com/artifact/com.datastax.spark/spark-cassandra-connector
https://github.com/datastax/spark-cassandra-connector#documentation

Для того, чтобы записать данные в cassandra, нам необходимо создать keyspace, используя утилиту `cqlsh`:
```shell
CREATE KEYSPACE IF NOT EXISTS airports 
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 3};
```

Теперь нужно создать таблицу со схемой:

In [ ]:
val typesMap = Map("string" -> "text", "int" -> "int")

val primaryKey = "ident"

val ddlColumns = airports.schema.fields.map { x =>
    if(x.name == primaryKey) {
        s"${x.name} ${typesMap(x.dataType.simpleString)} PRIMARY KEY"
    }
    else {
        s"${x.name} ${typesMap(x.dataType.simpleString)}"
    }
}.mkString(",")

val ddlQuery = s"CREATE TABLE IF NOT EXISTS airports.codes ($ddlColumns);"

println(ddlQuery)

Настроим параметры подключения к БД:

In [ ]:
import org.apache.spark.sql.cassandra._

spark.conf.set("spark.cassandra.connection.host", "127.0.0.1")
spark.conf.set("spark.cassandra.output.consistency.level", "ANY")
spark.conf.set("spark.cassandra.input.consistency.level", "ONE")

val tableOpts = Map("table" -> "codes","keyspace" -> "airports")

Теперь мы можем записать датасет в БД:

In [ ]:
airports
    .write
    .format("org.apache.spark.sql.cassandra")
    .mode("append")
    .options(tableOpts)
    .save()

In [ ]:
val df = spark
  .read
  .format("org.apache.spark.sql.cassandra")
  .options(tableOpts)
  .load()

df.show(1, 200, true)

Скорость чтения ОЧЕНЬ сильно зависит от структуры таблицы и запроса. Если мы сделаем запрос по колонке `ident`, которая является ключом, то будет применена оптимизация `filter pushdown` и запрос отработает очень быстро.

In [ ]:
df.filter('ident === "22WV").explain(true)

spark.time { 
    df.filter('ident === "22WV").show(1, 200, true)
}

Если же мы сделаем запрос по другой колонке, то он не будет так же эффективен, хотя `filter pushdown` тоже отработает. Это происходит из-за того, что зная ключ, Cassandra знает, на каком хосте и где находятся данные. Когда мы фильтруем по колонке, которая не является ключом, БД приходится искать эти данные на всем кластере.

In [ ]:
df.filter('iso_region === "RU").explain(true)

spark.time {
    df.filter('iso_region === "RU").show(1, 200, true)
}

Если сделать запрос более сложным, то `filter pushdown` не отработает и Spark прочитает всю таблицу целиком:

In [ ]:
import org.apache.spark.sql.functions._

df.filter(lower('iso_region) === "ru").explain(true)

spark.time {
    df.filter(lower('iso_region) === "ru").show(1, 200, true)
}

Необходимо понимать, что структура данной таблицы не отражает реального паттерна работой с данной базой - редко ключом является только одна колонка. В реальности ключ является составным и состоит из нескольких колонок, что позволяет делать более сложные запросы, которые будут обрабатываться также быстро, как первый запрос с фильтрацией по колонке "ident"

### Выводы
- cassandra - одна из немногих БД, которая способна эффективно хранить большие объемы данных
- в cassandra структура таблицы формируется, исходя из запросов, которые будут выполняться, а не наоборот
- в данной БД данные обычно хранятся в денормализованном виде (если утрировать - то по таблице на каждый запрос)
- Spark отлично подходит для чтения и записи данных в cassandra, но ее придется настроить под данный профиль нагрузки

## PostgreSQL
Классическая РБД

### Преимущества:
- это PostgreSQL

### Недостатки
- нет их

### Запуск в docker
```shell
docker run --rm -p 5432:5432 --name test_postgre -e POSTGRES_PASSWORD=12345 postgres:latest
```

Подключение с помощью psql:
```shell
docker run -it --rm postgres psql -h host.docker.internal -U postgres
```

### Spark connector
https://mvnrepository.com/artifact/org.postgresql/postgresql
https://spark.apache.org/docs/latest/sql-data-sources-jdbc.html

Для работы с БД создадим database:
```shell
CREATE DATABASE airports
```
Подготовим DDL для создания таблицы:

In [ ]:
val typesMap = Map("string" -> "VARCHAR (100)", "int" -> "INTEGER")

val primaryKey = "ident"

val ddlColumns = airports.schema.fields.map { x =>
    if(x.name == primaryKey) {
        s"${x.name} ${typesMap(x.dataType.simpleString)} PRIMARY KEY"
    }
    else {
        s"${x.name} ${typesMap(x.dataType.simpleString)}"
    }
}.mkString(",")

val ddlQuery = s"CREATE TABLE IF NOT EXISTS codes ($ddlColumns);"

val jdbcUrl = "jdbc:postgresql://localhost/airports?user=postgres&password=12345"

println(ddlQuery)

Запишем данные в БД:

In [ ]:
airports.write.format("jdbc").option("url", jdbcUrl).option("dbtable", "codes").mode("append").save

In [ ]:
val df = spark
    .read
    .format("jdbc")
    .option("url", jdbcUrl)
    .option("dbtable", "codes")
    .load()

df.printSchema
df.show(2, 200, true)

При использовании параметром по умолчанию мы получаем всего 1 партицию в DF:

In [ ]:
df.rdd.getNumPartitions

Исправить это можно, используя параметры `partitionColumn`, `lowerBound`, `upperBound`, `numPartitions`. Для этого нам понадобится добавиь новую колонку в нашу таблицу:

In [ ]:
val ddlColumns = airports.schema.fields.map { x =>
    if(x.name == primaryKey) {
        s"${x.name} ${typesMap(x.dataType.simpleString)} PRIMARY KEY"
    }
    else {
        s"${x.name} ${typesMap(x.dataType.simpleString)}"
    }
} :+ "id INTEGER" mkString(",")

val ddlQuery = s"CREATE TABLE IF NOT EXISTS codes_x ($ddlColumns);"

println(ddlQuery)

Перезапишем данные в новую таблицу:

In [ ]:
import org.apache.spark.sql.functions._

airports
    .withColumn("id", round(rand() * 10000).cast("int"))
    .write
    .format("jdbc")
    .option("url", jdbcUrl)
    .option("dbtable", "codes_x")
    .mode("append").save

Прочитаем таблицу, установив дополнительные параметры:

In [ ]:
val df = spark
    .read
    .format("jdbc")
    .option("url", jdbcUrl)
    .option("dbtable", "codes_x")
    .option("partitionColumn", "id")
    .option("lowerBound", "0")
    .option("upperBound", "10000")
    .option("numPartitions", "200")
    .load()

df.printSchema
df.show(2, 200, true)

Проверим, сколько партиций получилось:

In [ ]:
df.rdd.getNumPartitions

Проверим распределение данных по партициям:

In [ ]:
import org.apache.spark.sql.functions._

df.groupBy(spark_partition_id()).count().show(200, false)

### Выводы:
- Spark позволяет работать с PostgreSQL через JDBC коннектор
- При использовании `jdbc` настройка партиционирования задается вручную